In [1]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd

In [2]:
wrf_folder = 'M02_EnergyPlus_Forcing_Historical'
result_folder = 'eplus-results-base-csv'
sorted_folder = 'eplus-grid-emission-results'

In [3]:
with open(os.path.join('bldg-wrf-mapping/bldg-info-map.json'), 'r') as f:
    data = json.load(f)

In [4]:
data['420202853242'] # WRF_ID, BLDG_TYPE, TOTAL_FLOOR_AREA, YR_BT, NO_STORIES, FOOTPRINT_AREA

[31757, 'LSF', 18071.835679009753, 1992, 3, 6023.945226336585]

In [7]:
wrf_info_map = dict()
with open(os.path.join('bldg-wrf-mapping','wrf-grids-2229-w-bldgs.geojson'), 'r') as f:
    wrf_data = json.load(f)
    for wrf_grid in wrf_data['features']:
        if wrf_grid['properties']['has_bldgs'] == 1:
            wrf_id = wrf_grid['properties']['id']
            wrf_type_area = {}
            wrf_type_floor_area = {}
            for bldg_id in wrf_grid['properties']['bldg_ids']:
                if bldg_id in data:
                    if data[bldg_id][1] in wrf_type_area:
                        
                        wrf_type_area[data[bldg_id][1]] += data[bldg_id][2]
                        wrf_type_floor_area[data[bldg_id][1]] += data[bldg_id][5]
                    else:
                        wrf_type_area[data[bldg_id][1]] = data[bldg_id][2]
                        wrf_type_floor_area[data[bldg_id][1]] = data[bldg_id][5]
            wrf_info_map[wrf_id] = dict()
            wrf_info_map[wrf_id]['area'] = wrf_type_area
            wrf_info_map[wrf_id]['floor_area'] = wrf_type_floor_area

In [8]:
wrf_info_map[32790] # Area of each type in each WRF grid

{'area': {'LSF': 607312.6654331736,
  'SSF': 22863.209294897755,
  'MF': 8883.251734780984,
  None: 4776.8175525345505},
 'floor_area': {'LSF': 334873.1766913854,
  'SSF': 17159.66749151381,
  'MF': 4441.625867390492,
  None: 2388.4087762672752}}

In [9]:
geojson = open(os.path.join('bldg-wrf-mapping','wrf_to_bldg_info.json'), 'w')
geojson.write(json.dumps(wrf_info_map, indent=2))
geojson.close()

In [10]:
type_tags = ['SSF', 'LSF', 'MF', 'LOffice', 'MOffice', 'SOffice', 'SRetail', 'LRetail', 
             'SHotel', 'LHotel', 'Hospital', 'SRestaurant', 'LRestaurant', 'School', 'Warehouse']
count = 0
for grid_id in wrf_info_map:
    count += 1
    if count % 100 == 0:
        sys.stdout.write("\rTotal grid processed: %d" % count)
        sys.stdout.flush()
    df_emission_result = pd.DataFrame()
    df_energy_result = pd.DataFrame()
    for type_tag in type_tags:
        result_name = os.path.join(wrf_folder, result_folder, str(grid_id) + '-' + type_tag + '-out.csv')
        if os.path.exists(result_name):
            df_eplus_result = pd.read_csv(result_name, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
            
            col_list = ['Environment:Site Total Zone Exfiltration Heat Loss [J](Hourly)',
                        'Environment:Site Total Zone Exhaust Air Heat Loss [J](Hourly)',
                        'SimHVAC:Air System Relief Air Total Heat Loss Energy [J](Hourly)',
                        'SimHVAC:HVAC System Total Heat Rejection Energy [J](Hourly)']
            col_list_1 = ['Environment:Site Total Zone Exfiltration Heat Loss [J](Hourly)',
                          'Environment:Site Total Zone Exhaust Air Heat Loss [J](Hourly)']
            col_list_2 = 'SimHVAC:Air System Relief Air Total Heat Loss Energy [J](Hourly)'
            col_list_3 = 'SimHVAC:HVAC System Total Heat Rejection Energy [J](Hourly)'

#           col_list = ['Environment:Site Total Surface Heat Emission to Air [J](Hourly)'
            df_emission_result[str(grid_id) + '-' + type_tag.lower()] = df_eplus_result[col_list].sum(axis=1) * 1E-9
            df_emission_result[str(grid_id) + '-' + type_tag.lower() + '-exh'] = df_eplus_result[col_list_1].sum(axis=1) * 1E-9
            df_emission_result[str(grid_id) + '-' + type_tag.lower() + '-ref'] = df_eplus_result[col_list_2] * 1E-9
            df_emission_result[str(grid_id) + '-' + type_tag.lower() + '-rej'] = df_eplus_result[col_list_3] * 1E-9
                
#             try:
#                 col_list_e = ['Electricity:HVAC [J](Hourly)',
#                               'NaturalGas:HVAC [J](Hourly)']
#                 df_energy_result[str(grid_id) + '-' + type_tag.lower() + '-energy'] = df_eplus_result[col_list_e].sum(axis=1) * 1E-9
#             except:
#                 col_list_e = ['Electricity:HVAC [J](Hourly)']
#                 df_energy_result[str(grid_id) + '-' + type_tag.lower() + '-energy'] = df_eplus_result[col_list_e].sum(axis=1) * 1E-9
#         else:
#             print(result_name)
    df_emission_result.to_csv(os.path.join(wrf_folder, sorted_folder, 'eplus-emission-result-' + str(grid_id) + '.csv'))
#     df_energy_result.to_csv(os.path.join(wrf_folder, sorted_folder, 'eplus-energy-result-' + str(grid_id) + '.csv'))

Total grid processed: 5800

In [11]:
df_emission_result.head(12)

,53110-ssf,53110-ssf-exh,53110-ssf-ref,53110-ssf-rej,53110-lsf,53110-lsf-exh,53110-lsf-ref,53110-lsf-rej
0,0.007158,0.007158,0.0,0.0,0.007019,0.007019,0.0,0.0
1,0.006422,0.006422,0.0,0.0,0.006346,0.006346,0.0,0.0
2,0.007350,0.007350,0.0,0.0,0.007592,0.007592,0.0,0.0
3,0.009085,0.009085,0.0,0.0,0.009805,0.009805,0.0,0.0
4,0.008849,0.008849,0.0,0.0,0.009645,0.009645,0.0,0.0
5,0.008056,0.008056,0.0,0.0,0.009027,0.009027,0.0,0.0
6,0.001737,0.001737,0.0,0.0,0.002420,0.002420,0.0,0.0
7,0.001220,0.001220,0.0,0.0,0.001726,0.001726,0.0,0.0
8,0.000756,0.000756,0.0,0.0,0.001048,0.001048,0.0,0.0
9,0.000705,0.000705,0.0,0.0,0.000894,0.000894,0.0,0.0


## Calc into grid

In [12]:
type_tags = {'SSF':235.93, 'LSF': 268.58, 'MF': 646.60, 'LOffice': 46320.38, 'MOffice':4982.19, 
             'SOffice':511.16, 'SRetail':2090.32, 'LRetail':2293.99, 
             'SHotel': 4013.58, 'LHotel': 11345.29, 'Warehouse': 4835.13, 
             'SRestaurant': 511.15, 'LRestaurant': 511.15, 
             'School': 19592.00, 'Hospital': 22436.18, 'None': 268.58} # unit:m2
wrf_info_map[32790]

{'area': {'LSF': 607312.6654331736,
  'SSF': 22863.209294897755,
  'MF': 8883.251734780984,
  None: 4776.8175525345505},
 'floor_area': {'LSF': 334873.1766913854,
  'SSF': 17159.66749151381,
  'MF': 4441.625867390492,
  None: 2388.4087762672752}}

In [14]:
num_of_days = 41
df_grid_result = pd.DataFrame()
df_grid_result_exh = pd.DataFrame()
df_grid_result_ref = pd.DataFrame()
df_grid_result_rej = pd.DataFrame()
for grid_id in wrf_info_map:
    df_grid_result[str(grid_id)] = np.zeros(num_of_days * 24)
    df_grid_result_exh[str(grid_id)] = np.zeros(num_of_days * 24)
    df_grid_result_ref[str(grid_id)] = np.zeros(num_of_days * 24)
    df_grid_result_rej[str(grid_id)] = np.zeros(num_of_days * 24)

count = 0
for grid_id in wrf_info_map:
    count += 1
#     if count % 100 == 0:
#         sys.stdout.write("\rTotal grid processed: %d" % count)
#         sys.stdout.flush()
    csv_name = os.path.join(wrf_folder, sorted_folder, 'eplus-emission-result-' + str(grid_id) + '.csv')
    if not os.path.exists(csv_name):
        continue
    df_emission_result = pd.read_csv(csv_name, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
    total_floor_area = 0
    for use_type in wrf_info_map[grid_id]['area']:
        if use_type:
            header_name = str(grid_id) + '-' + use_type.lower()
            bldg_area = wrf_info_map[grid_id]['area'][use_type]
            floor_area = wrf_info_map[grid_id]['floor_area'][use_type]
            if header_name in df_emission_result:
                df_grid_result[str(grid_id)] += (df_emission_result[header_name] * 277778 / type_tags[use_type]) * bldg_area
                df_grid_result_exh[str(grid_id)] += (df_emission_result[header_name + '-exh'] * 277778 / type_tags[use_type]) * bldg_area 
                df_grid_result_ref[str(grid_id)] += (df_emission_result[header_name + '-ref'] * 277778 / type_tags[use_type]) * bldg_area 
                df_grid_result_rej[str(grid_id)] += (df_emission_result[header_name + '-rej'] * 277778 / type_tags[use_type]) * bldg_area
                total_floor_area += floor_area
            else:
                print(str(grid_id) + ' ' + use_type + ' - not found.')
    df_grid_result[str(grid_id)] /= total_floor_area
    df_grid_result_exh[str(grid_id)] /= total_floor_area
    df_grid_result_ref[str(grid_id)] /= total_floor_area
    df_grid_result_rej[str(grid_id)] /= total_floor_area


24971 SHotel - not found.
25343 SSF - not found.
28409 SSF - not found.
29361 SHotel - not found.
32768 School - not found.
32768 SRetail - not found.
32768 LRetail - not found.
32768 SRestaurant - not found.
32768 LRestaurant - not found.
32768 MOffice - not found.
32768 Hospital - not found.
32768 LOffice - not found.
32768 MF - not found.
32768 LSF - not found.
35201 MF - not found.
39575 SRetail - not found.
39812 SRestaurant - not found.
40207 SRetail - not found.
42916 Hospital - not found.


In [16]:
df_grid_result[str(53110)].head(12)

0     13.762248
1     12.434107
2     14.828136
3     19.089576
4     18.766044
5     17.530618
6      4.637132
7      3.304046
8      2.009116
9      1.723091
10     2.583347
11     3.684354
Name: 53110, dtype: float64

In [17]:
df_grid_result.to_csv(os.path.join(wrf_folder, 'eplus-final-results', 'grid-emission-results-new.csv'))
df_grid_result_exh.to_csv(os.path.join(wrf_folder, 'eplus-final-results', 'grid-emission-results-exh-new.csv'))
df_grid_result_ref.to_csv(os.path.join(wrf_folder, 'eplus-final-results', 'grid-emission-results-ref-new.csv'))
df_grid_result_rej.to_csv(os.path.join(wrf_folder, 'eplus-final-results', 'grid-emission-results-rej-new.csv'))

In [18]:
df_grid_result = pd.read_csv(os.path.join(wrf_folder, 'eplus-final-results', 'grid-emission-results-rej-new.csv'), sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

In [19]:
df_grid_result[str(32790)].to_list()[:12]

[0.011796655392696677,
 0.004903829236742607,
 0.0007548261222055361,
 0.0,
 0.0018550328632249641,
 0.006589650307343186,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [20]:
file_name_prefix = "Variable_HEATI_M0_2018" ## todo
output_folder = 'eplus-final-results-matrix'
no_row = 384
no_col = 339
for index, row in df_grid_result.iterrows():
    hour = index % 24
    day = (index // 24 + 20) % 31
    mon = (index // 24 + 20) // 31 + 8
    if day == 0:
        day = 31
        mon = 8
    file_name = file_name_prefix + str(mon).zfill(2) + str(day).zfill(2) + str(hour).zfill(2) + '.txt'
    ind = -1
    with open(os.path.join(wrf_folder, output_folder, file_name), 'w') as out:
        for i in range(no_row):
            for j in range(no_col):
                ind += 1
                if str(ind) not in row:
                    out.write('0,')
                else:
                    out.write(str(round(row[str(ind)], 3)) + ',')
            out.write('\n')